In [170]:
%load_ext autoreload
%autoreload 2
import utils.dataset as dataset
import utils.preprocessing as preprocessing
from utils.logger import Logger
import lstm.model as model
import utils.postprocessing as postprocessing
import pandas as pd
from sklearn import metrics
from numpy import linalg as LA
import numpy as np
import logging
import argparse


from astropy.convolution import Gaussian1DKernel, convolve

import torch
import torch.nn as nn
from pathlib import Path
from torch.autograd import Variable

np.random.seed(777)
torch.manual_seed(777)
torch.cuda.manual_seed_all(777)

print(torch.rand(2))
print("777 is {}, {}".format(0.0819, 0.4911))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
tensor([0.0819, 0.4911])
777 is 0.0819, 0.4911


In [34]:
import torch
import torch.nn as nn
from pathlib import Path
from torch.autograd import Variable

In [28]:
train_path= '../../DATA/SWaT/SWaT_Physical/SWaT_Dataset_Normal_v0.csv'
test_path = '../../DATA/SWaT/SWaT_Physical/SWaT_Dataset_Attack_v0.csv'

train_x, test_x, test_y = dataset.dataset(train_path = train_path, test_path = test_path)

In train : {'Normal': 496800}
In test : {'Attack': 54621, 'Normal': 395298}


# SVM

In [27]:
from sklearn import svm

In [52]:
# comp_num = 45
clf = svm.OneClassSVM(nu=0.001, kernel="rbf", gamma=0.0001, verbose=True)
clf.fit(train_x[0:20000,0:2])
preds = clf.predict(test_x[5000:15000, 0:2]) 
unique, counts = np.unique(preds, return_counts=True)
print('predict 빈도:', dict(zip(unique, counts)))

[LibSVM]predict 빈도: {-1: 4842, 1: 5158}


In [46]:
# comp_num = 45
clf = svm.OneClassSVM(nu=0.001, kernel="rbf", gamma=0.0001, verbose=True)
clf.fit(train_x[0:20000,0:4])
preds = clf.predict(test_x[5000:15000, 0:4]) 
unique, counts = np.unique(preds, return_counts=True)
print('predict 빈도:', dict(zip(unique, counts)))

[LibSVM]predict 빈도: {-1: 2390, 1: 7610}


In [43]:
# comp_num = 2
clf = svm.OneClassSVM(nu=0.001, kernel="rbf", gamma=0.0001, verbose=True)
clf.fit(train_x[0:20000,:])
preds = clf.predict(test_x[5000:15000, :]) 
unique, counts = np.unique(preds, return_counts=True)
print('predict 빈도:', dict(zip(unique, counts)))

[LibSVM]predict 빈도: {-1: 4842, 1: 5158}


# LSTM

## model

In [5]:
class ENCODER(nn.Module):

    def __init__(self,args):
        super(ENCODER, self).__init__()
        self.args = args
        self.drop = nn.Dropout(args['dropout'])
        self.linear = nn.Linear(args['hidden_size'], args['data_dim'])

        if args['cell_type'] in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, args['cell_type'])(args['rnn_inp_size'], args['hidden_size'], args['nlayers'], dropout=args['dropout'])

    def init_weights(self):
        initrange = 0.1
        self.linear.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.fill_(0)

    def forward(self, input, hidden, return_hiddens=False, noise=False):
        output, hidden = self.rnn(input, hidden)
        output = self.linear(output.contiguous().view(-1,self.args['hidden_size']))
        output = output.contiguous().view(input.size()[0], -1, self.args['rnn_inp_size'])
        return output, hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data ############# 이게 무엇
        if self.args['cell_type'] == 'LSTM':
            return (Variable(weight.new(self.args['nlayers'], bsz, self.args['hidden_size']).zero_()),
                    Variable(weight.new(self.args['nlayers'], bsz, self.args['hidden_size']).zero_()))

    def repackage_hidden(self,h):
        """Wraps hidden states in new Variables, to detach them from their history."""
        if type(h) == tuple:
            return tuple(self.repackage_hidden(v) for v in h)
        else:
            return Variable(h.data)

    def extract_hidden(self, hidden):
        if self.args['cell_type'] == 'LSTM':
            return hidden[0][-1].data.cpu()  # hidden state last layer (hidden[1] is cell state)
        else:
            return hidden[-1].data.cpu()  # last layer

        
class DECODER(nn.Module):

    def __init__(self,args):
        super(DECODER, self).__init__()
        self.args = args
        self.drop = nn.Dropout(args['dropout'])
        self.linear = nn.Linear(args['hidden_size'], args['data_dim'])

        if args['cell_type'] in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, args['cell_type'])(args['rnn_inp_size'], args['hidden_size'], args['nlayers'], dropout=args['dropout'])


    def init_weights(self):
        initrange = 0.1
        self.linear.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.fill_(0)
        
    def forward(self, input, hidden, return_hiddens=False, noise=False):
        
        output, hidden = self.rnn(input, hidden)
        output = self.linear(output.contiguous().view(-1,self.args['hidden_size']))
        output = output.contiguous().view(input.size()[0], -1, self.args['rnn_inp_size'])

        return output, hidden




    def init_hidden(self, bsz):
        weight = next(self.parameters()).data ############# 이게 무엇
        if self.args['cell_type'] == 'LSTM':
            return (Variable(weight.new(self.args['nlayers'], bsz, self.args['hidden_size']).zero_()),
                    Variable(weight.new(self.args['nlayers'], bsz, self.args['hidden_size']).zero_()))

    def repackage_hidden(self,h):
        """Wraps hidden states in new Variables, to detach them from their history."""
        if type(h) == tuple:
            return tuple(self.repackage_hidden(v) for v in h)
        else:
            return Variable(h.data)

    def extract_hidden(self, hidden):
        if self.args['cell_type'] == 'LSTM':
            return hidden[0][-1].data.cpu()  # hidden state last layer (hidden[1] is cell state)
        else:
            return hidden[-1].data.cpu()  # last layer

## solver

In [136]:
check_step = 100
endPoint = 449916


In [150]:
solver.anomal_score

array([0.01908201, 0.01776421, 0.01737914, ..., 0.01660304, 0.01747271,
       0.01548675], dtype=float32)

In [151]:
for conv in range(0, 50, 10):
    print(conv)

0
10
20
30
40


In [154]:
max_f1 = 0

anomal_score = LA.norm(anomal_score, axis=1)

for conv in range(0, 100, 10):
    if conv != 0:
        gauss_kernel = Gaussian1DKernel(conv)
        norm = convolve(anomal_score, gauss_kernel)
    else:
        norm = anomal_score

    precision, recall, thresholds = metrics.precision_recall_curve(  test_y[:449916].cpu().numpy(), norm ,pos_label =1)

    beta = 1
    f1 = (1+beta**2)*(precision*recall)/((beta**2*precision)+recall)
    f1 = np.nan_to_num(f1)

    max_pre_tp = precision[np.argmax(f1)]
    max_recall_tp = recall[np.argmax(f1)]
    max_f1_tp = f1[np.argmax(f1)]

    attack_list_acc = []
    anomaly = (norm > thresholds[np.argmax(f1)] )
    for idxx in range(0, len(attack_list)):
        k, t = np.unique(anomaly[attack_list['Start Time'][idxx]:attack_list['End Time'][idxx]], return_counts=True)
        if t.shape[0] > 1:
            attack_list_acc.append(t[1]/(t[0] + t[1]))
        elif k == 1:
            attack_list_acc.append(1.0)
        elif k == 0:
            attack_list_acc.append(0.0)
        else:
            print("faral error in evaluation")

    zerolist = 0
    for k in range(len(attack_list_acc)):
        if attack_list_acc[k] == 0.0:
            zerolist = zerolist + 1  
            
    if max_f1 < max_f1_tp:
        max_pre = max_pre_tp
        max_recall = max_recall_tp
        max_f1 = max_f1_tp
        max_zerolist = zerolist
        max_conv = conv

print("conv[{}]\t precision[{}]\t recall[{}]\t f1[{}] \t find attack [{}]".format(max_conv, max_pre, max_recall, max_f1, 36 - max_zerolist))

epoch[0]	 precision[0.9245685161589251]	 recall[0.7306530455319382]	 f1[0.8162518151882684] 	 find attack [17]


In [142]:
thresholds[np.argmax(f1)]

0.034600586

In [144]:
attack_list

,Attack number,Start Time,End Time,Attack Point,Start State,Attack,Actual Change,Expected Impact or attacker intent,pointattack_SS_SM_MS_MM
0,1,1754,2694,MV-101,MV-101 is closed,Open MV-101,Yes,Tank overflow,SS
1,2,3068,3511,P-102,P-101 is on where as P-102 is off,Turn on P-102,Yes,Pipe bursts,SS
2,3,4920,5303,LIT-101,Water level between L and H,Increase by 1 mm every second,No,Tank Underflow; Damage P-101,SS
3,4,6459,6849,MV-504,MV-504 is closed,Open MV-504,Yes,Halt RO shut down sequence; Reduce life of RO,SS
4,6,7255,7451,AIT-202,Value of AIT-202 is >7.05,Set value of AIT-202 as 6,No,P-203 turns off; Change in water quality,SS
5,7,7705,8134,LIT-301,Water level between L and H,Water level increased above HH,No,Stop of inflow; Tank underflow; Damage P-301,SS
6,8,11410,12374,DPIT-301,Value of DPIT is <40kpa,Set value of DPIT as >40kpa,No,Backwash process is started again and again; N...,SS
7,10,15380,15540,FIT-401,Value of FIT-401 above 1,Set value of FIT-401 as <0.7,No,UV shutdown; P-501 turns off;,SS
8,11,15541,16101,FIT-401,Value of FIT-401 above 1,Set value of FIT-401 as 0,No,UV shutdown; P-501 turns off;,SS
9,13,73800,74521,MV-304,MV-304 is open,Close MV-304,Yes,Halt of stage 3 because change in the backwash...,SS


In [145]:
attack_list_acc = []
anomaly = (norm > thresholds[np.argmax(f1)] )
for idxx in range(0, len(attack_list)):
    k, t = np.unique(anomaly[attack_list['Start Time'][idxx]:attack_list['End Time'][idxx]], return_counts=True)
    if t.shape[0] > 1:
        attack_list_acc.append(t[1]/(t[0] + t[1]))
    elif k == 1:
        attack_list_acc.append(1.0)
    elif k == 0:
        attack_list_acc.append(0.0)
    else:
        print("faral error in evaluation")

In [146]:
attack_list_acc

[0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.011583011583011582,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.9986159169550173,
 0.8910306406685237,
 0.0,
 0.9803586678052946,
 0.0,
 0.0,
 0.0,
 0.8316831683168316,
 0.0,
 0.0,
 0.8997867803837953,
 0.0,
 0.9600997506234414,
 0.9528619528619529,
 0.0]

In [ ]:
precision, recall, thresholds = metrics.precision_recall_curve(  self.test_y[:449916].cpu().numpy(), self.anomal_score,pos_label =1)

maximum = norm.max()
th = np.linspace(0, maximum, num_samples)
    attack_list_acc = []
    for idxx in range(0, len(attack_list)):
        k, t = np.unique(anomaly[attack_list['Start Time'][idxx]:attack_list['End Time'][idxx]], return_counts=True)
        if t.shape[0] > 1:
            attack_list_acc.append(t[1]/(t[0] + t[1]))
        elif k == 1:
            attack_list_acc.append(1.0)
        elif k == 0:
            attack_list_acc.append(0.0)
        else:
            print("faral error in evaluation")
    attack_list_.append(attack_list_acc)

In [162]:
postprocessing.evaluate_conv(solver.anomal_score, solver.test_y, attack_list)

conv[40]	 precision[0.9245685161589251]	 recall[0.7306530455319382]	 f1[0.8162518151882684] 	 find attack [17]


(40, 0.9245685161589251, 0.7306530455319382, 0.8162518151882684, 19)

In [163]:
class Solver():
    def __init__(self, args):
        
        torch.manual_seed(777)
        torch.cuda.manual_seed_all(777)

        self.attack_list = pd.read_csv(args.attack_list_path, error_bad_lines=False, sep='\t')

        self.tf_log = args.tf_log
        
        train_x, test_x, test_y = dataset.dataset(train_path = args.train_path, test_path = args.test_path)
        
        train_x_batchfy = preprocessing.batchify(args, train_x, args.batch_size)
        test_x_batchfy = preprocessing.batchify(args, test_x, args.batch_size)
        generate_batchfy = preprocessing.batchify(args, test_x, 1)
        train_generate_batchfy = preprocessing.batchify(args, train_x, 1)
        
        self.train_x_batchfy = train_x_batchfy[:,:,args.selected_dim]
        self.test_x_batchfy = test_x_batchfy[:,:,args.selected_dim]
        self.generate_batchfy = generate_batchfy[:,:,args.selected_dim]
        self.train_generate_batchfy = train_generate_batchfy[:,:,args.selected_dim]
        self.test_y = test_y
        

        self.args = args
        self.encoder = model.ENCODER(self.args)
        self.encoder.cuda()

        self.decoder = model.DECODER(self.args)
        self.decoder.cuda()

        self.optim_enc   = torch.optim.Adam(self.encoder.parameters(), self.args.lr)
        self.optim_dec   = torch.optim.Adam(self.decoder.parameters(), self.args.lr)

        self.loss_fn = nn.MSELoss()    
    
        self.logger = Logger('./tf_logs')
    
        self.base_dir = Path('model_save')
        self.base_dir.mkdir(parents=True,exist_ok=True)      
        
#         self.evaluate = args.evaluate
    
#     def make_dir_name(self, args):
#         return 'modelName:'+args.model_name+'__cellType:'+args.cell_type \
#                 + '__hidSize:' + str(args.hidden_size) + '__dropout:' + str(args.dropout)

    def load(self, path):
        try:
#             checkpoint = torch.load(Path(self.base_dir, str(args.selected_dim)))
#             checkpoint = checkpoint.with_suffix('.pth')
#             start_epoch = checkpoint['epoch
#             self.encoder.load_state_dict(checkpoint['state_dict_enc)
#             self.optim_enc.load_state_dict((checkpoint['optimizer_enc))
#             self.decoder.load_state_dict(checkpoint['state_dict_dec)
#             self.optim_dec.load_state_dict((checkpoint['optimizer_dec))            
#             del checkpoint
            print("=> loaded checkpoint")
        except:
            print("=> Not exist checkpoint")
            pass        

    def fit(self, load):
        total_loss = 0
        max_f1 = 0
        total_length = self.train_x_batchfy.size(0) - 1
        
        
        log = logging.getLogger('snowdeer_log')
        log.setLevel(logging.DEBUG)

        formatter = logging.Formatter('[%(levelname)s] (%(filename)s:%(lineno)d) > %(message)s')

        fileHandler = logging.FileHandler('./log.txt')

        fileHandler.setFormatter(formatter)
        log.addHandler(fileHandler)
        
        log.critical(str(args.selected_dim))
        
                                          
        for epoch in range(0, self.args.epoch):

            self.encoder.train()
            self.decoder.train()
                
            hidden_enc = self.encoder.init_hidden(self.args.batch_size)

            for batch, i in enumerate(range(0, self.train_x_batchfy.size(0) - 1, self.args.seq_length)):
                outSeq = []
                inputSeq, targetSeq = preprocessing.get_batch(self.args, self.train_x_batchfy, i)

                if args.seq_length != targetSeq.size()[0] :
                    continue
                hidden_enc = self.encoder.repackage_hidden(hidden_enc)
                self.optim_enc.zero_grad()
                self.optim_dec.zero_grad()
                
                Outputseq_enc, hidden_enc = self.encoder.forward(inputSeq, hidden_enc, return_hiddens=True)
                deccoder_input = Variable(torch.zeros(Outputseq_enc.size())).cuda()
                
                deccoder_input[0,:,:] = Outputseq_enc[-1,:,:] # inputSeq[-1,:,:]
                deccoder_input[1:,:,:] = targetSeq[:-1,:,:]
                
                loss_enc = self.loss_fn(Outputseq_enc[-1,:,:].view(self.args.batch_size, -1), targetSeq[0,:,:].contiguous().view(self.args.batch_size, -1))
                loss_enc.backward(retain_graph=True)
                torch.nn.utils.clip_grad_norm_(self.encoder.parameters(), self.args.clip)
                
                self.optim_enc.step()     
                
                Outputseq_enc, hidden_enc = self.decoder.forward(deccoder_input, hidden_enc, return_hiddens=True)
                loss_dec = self.loss_fn(Outputseq_enc.view(args.batch_size, -1), targetSeq.contiguous().view(args.batch_size, -1))   
                loss_dec.backward()
                torch.nn.utils.clip_grad_norm_(self.decoder.parameters(), self.args.clip)
                self.optim_dec.step()
                
                total_loss += loss_enc.item() + loss_dec.item()        

                if batch % 30 == 0 and self.tf_log == True :
                    # 1. Log scalar values (scalar summary)
                    info = { 'enc_loss': loss_enc.item(), 'dec_loss' : loss_dec.item() }

                    for tag, value in info.items():
                        self.logger.scalar_summary(tag, value, epoch*total_length + i +1)

                    # 2. Log values and gradients of the parameters (histogram summary)
                    for tag, value in self.encoder.named_parameters():
                        tag = tag.replace('.', '/')
                        self.logger.histo_summary(tag, value.data.cpu().numpy(), epoch*total_length + i +1)
                        
                    for tag, value in self.decoder.named_parameters():
                        tag = tag.replace('.', '/')
                        self.logger.histo_summary(tag, value.data.cpu().numpy(), epoch*total_length + i +1)
            
            total_loss = 0    
            self.anomal_score = postprocessing.get_anomalscore_encdec(base_model = self,  \
                                                                 generate_batchfy = self.generate_batchfy, length = 449916,args = self.args)
#             self.anomal_score = postprocessing.get_anomalscore_minmax(base_model = self,  \
#                                                                  generate_batchfy = self.generate_batchfy, length = 449916,args = self.args)

            self.anomal_score = LA.norm(self.anomal_score, axis=1)
            max_conv, max_pre, max_recall, max_f1_tp, max_zerolist = postprocessing.evaluate_conv(self.anomal_score, self.test_y, self.attack_list)
    
            print("epoch[{}]\t conv[{}]\t precision[{}]\t recall[{}]\t f1[{}]\t findnum[{}]".format(epoch, max_conv, max_pre, max_recall, max_f1_tp,36 - max_zerolist))
            log.info("epoch[{}]\t conv[{}]\t precision[{}]\t recall[{}]\t f1[{}]\t findnum[{}]".format(epoch, max_conv, max_pre, max_recall, max_f1_tp,36 - max_zerolist))
            
            if max_f1_tp > max_f1:
                self.model_dictionary = {'epoch': epoch + 1,
                        'state_dict_enc': self.encoder.state_dict(),
                        'optimizer_enc': self.optim_enc.state_dict(),
                        'state_dict_dec': self.decoder.state_dict(),
                        'optimizer_dec': self.optim_dec.state_dict(),
                        'args':args,
                        'loss':total_loss,
                        'anomal_score' : self.anomal_score 
                        }
                self.save_checkpoint(self.args, self.model_dictionary)                                    
                                    
            
            
            
 
    def save_checkpoint(self, args, state):
        checkpoint = Path(self.base_dir, str(args.selected_dim))
        checkpoint = checkpoint.with_suffix('.pth')
        torch.save(state, checkpoint)
        

In [166]:
attack_list = '../../DATA/SWaT/SWaT_Physical/attack_list.csv'
attack_list = pd.read_csv(attack_list, error_bad_lines=False, sep='\t')

In [30]:
train_path= '../../DATA/SWaT/SWaT_Physical/SWaT_Dataset_Normal_v0.csv'
test_path = '../../DATA/SWaT/SWaT_Physical/SWaT_Dataset_Attack_v0.csv'

attack_list = '../../DATA/SWaT/SWaT_Physical/attack_list.csv'
attack_list = pd.read_csv(attack_list, error_bad_lines=False, sep='\t')

args={}
args['tf_log'] = False
args['model_name'] = 'enc_dec'
args['cuda'] = True
args['batch_size'] = 256
args['clip'] = 1
args['train_path'] = train_path
args['test_path'] = test_path
args['dropout'] = 0.5
args['hidden_size'] = 32
args['nlayers'] = 2
args['lr'] =0.001
args['cell_type'] = 'LSTM'
args['epoch'] = 3
args['seq_length'] = 2
args['data_dim'] = 4
args['selected_dim'] = [36,38,28,40]
args['rnn_inp_size'] = args['data_dim']
args['evaluate'] = True

In [167]:
parser = argparse.ArgumentParser()

parser.add_argument("--cuda", action="store_true")
parser.add_argument("--tf_log", action="store_false")
parser.add_argument("--model_name", type=str, default="enc_dec")
parser.add_argument("--batch_size", type=int, default=256)
parser.add_argument("--clip", type=int, default=1)

parser.add_argument("--train_path", type=str, default="../../DATA/SWaT/SWaT_Physical/SWaT_Dataset_Normal_v0.csv")
parser.add_argument("--test_path", type=str, default="../../DATA/SWaT/SWaT_Physical/SWaT_Dataset_Attack_v0.csv")
parser.add_argument("--attack_list_path", type=str, default='../../DATA/SWaT/SWaT_Physical/attack_list.csv')

parser.add_argument("--dropout", type=float, default=0.5)
parser.add_argument("--hidden_size", type=int, default=32)
parser.add_argument("--nlayers", type=int, default=2)
parser.add_argument("--lr", type=float, default=0.001)

parser.add_argument("--cell_type", type=str, default="LSTM")
parser.add_argument("--epoch", type=int, default=5)
parser.add_argument("--seq_length", type=int, default=2)
parser.add_argument('--selected_dim', nargs='+', type=int, default=[36, 38, 28, 40])
args = parser.parse_args([])

In [168]:
solver = Solver(args = args)

tensor([0.0819, 0.4911])
777 is 0.0819, 0.4911
In train : {'Normal': 496800}
In test : {'Attack': 54621, 'Normal': 395298}
data length is 51


In [171]:
solver.fit(load = False)
# data drop == 51

anomal_detection...
epoch[0]	 conv[10]	 precision[0.978645752446032]	 recall[0.6922062942824189]	 f1[0.8043803781420329]	 findnum[12]
anomal_detection...
epoch[1]	 conv[20]	 precision[0.9601905082625742]	 recall[0.7308178173230077]	 f1[0.8043803781420329]	 findnum[17]


KeyboardInterrupt: 

In [127]:
solver.fit(load = False)
# data drop == 51

anomal_detection...
epoch[0]	 precision[0.9759514359094092]	 recall[0.6887460866699621]	 f1[0.8075734157650696]
anomal_detection...
epoch[1]	 precision[0.9846234446627374]	 recall[0.6881602314128266]	 f1[0.8101211259105996]
anomal_detection...
epoch[2]	 precision[0.9858296390120329]	 recall[0.6839677047289504]	 f1[0.8076137358539511]
anomal_detection...
epoch[3]	 precision[0.9880620562713647]	 recall[0.6879405356914008]	 f1[0.811129939234331]
anomal_detection...
epoch[4]	 precision[0.9806387642208537]	 recall[0.6880503835521137]	 f1[0.8086933132497715]


In [77]:
solver.fit(load = False)
# data drop == 51

anomal_detection...
epoch[0]	 precision[0.9852968156204953]	 recall[0.6882700792735394]	 f1[0.8104251099422264]
anomal_detection...
epoch[1]	 precision[0.9794525899111957]	 recall[0.6885630069021073]	 f1[0.8086433025155881]
anomal_detection...
epoch[2]	 precision[0.9830969700141294]	 recall[0.6878673037842588]	 f1[0.8094012214694256]
anomal_detection...
epoch[3]	 precision[0.9879065120803429]	 recall[0.6879588436681863]	 f1[0.8110902458503313]


In [97]:
solver.fit(load = False)
# data drop == 51

anomal_detection...
epoch[0]	 precision[0.9852968156204953]	 recall[0.6882700792735394]	 f1[0.8104251099422264]
anomal_detection...
epoch[1]	 precision[0.9794525899111957]	 recall[0.6885630069021073]	 f1[0.8086433025155881]
anomal_detection...
epoch[2]	 precision[0.9830969700141294]	 recall[0.6878673037842588]	 f1[0.8094012214694256]


In [80]:
solver.fit(load = False)
# data drop == 45

anomal_detection...
epoch[0]	 precision[0.9829493449096472]	 recall[0.6881419234360411]	 f1[0.8095412448847727]
anomal_detection...
epoch[1]	 precision[0.9778136135459203]	 recall[0.6882700792735394]	 f1[0.807882408561482]


KeyboardInterrupt: 

In [82]:
solver.fit(load = False)
# data drop == 45

anomal_detection...
epoch[0]	 precision[0.9829493449096472]	 recall[0.6881419234360411]	 f1[0.8095412448847727]
anomal_detection...
epoch[1]	 precision[0.9778136135459203]	 recall[0.6882700792735394]	 f1[0.807882408561482]
anomal_detection...
epoch[2]	 precision[0.983513315160411]	 recall[0.68697021292177]	 f1[0.8089206989102432]
anomal_detection...
epoch[3]	 precision[0.9867132316256597]	 recall[0.6879588436681863]	 f1[0.8106877804625475]
